# Sparse Autoencoder Training Demo

## Setup

In [1]:
# Autoreload
%load_ext autoreload
%autoreload 2

In [2]:
from sparse_autoencoder import TensorActivationStore, SparseAutoencoder, pipeline
from sparse_autoencoder.source_data.pile_uncopyrighted import PileUncopyrightedDataset
from transformer_lens import HookedTransformer
from transformer_lens.utils import get_device
from transformers import PreTrainedTokenizerBase
import torch
import wandb

In [3]:
device = get_device()

### Source Model

In [4]:
src_model = HookedTransformer.from_pretrained("solu-1l", dtype="float32")
src_d_mlp: int = src_model.cfg.d_mlp  # type: ignore
src_d_mlp

Loaded pretrained model solu-1l into HookedTransformer


2048

### Source Dataset

In [5]:
tokenizer: PreTrainedTokenizerBase = src_model.tokenizer  # type: ignore
source_data = PileUncopyrightedDataset(tokenizer=tokenizer)
src_dataloader = source_data.get_dataloader(batch_size=8)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Resolving data files:   0%|          | 0/30 [00:00<?, ?it/s]

### Activation Store

In [6]:
max_items = 1_000_000
store = TensorActivationStore(max_items, src_d_mlp, device)

### Autoencoder

In [7]:
autoencoder = SparseAutoencoder(src_d_mlp, src_d_mlp * 8, torch.zeros(src_d_mlp))
autoencoder

SparseAutoencoder(
  (encoder): Sequential(
    (0): TiedBias(position=TiedBiasPosition.PRE_ENCODER)
    (1): ConstrainedUnitNormLinear(in_features=2048, out_features=16384, bias=True)
    (2): ReLU()
  )
  (decoder): Sequential(
    (0): ConstrainedUnitNormLinear(in_features=16384, out_features=2048, bias=False)
    (1): TiedBias(position=TiedBiasPosition.POST_DECODER)
  )
)

## Training

If you initialise [wandb](https://wandb.ai/site), the pipeline will automatically log all metrics to wandb.

In [8]:
# wandb.init(project="sparse-autoencoder", dir=".cache/wandb")

In [9]:
pipeline(
    src_model=src_model,
    src_model_activation_hook_point="blocks.0.mlp.hook_post",
    src_model_activation_layer=0,
    src_dataloader=src_dataloader,
    activation_store=store,
    num_activations_before_training=max_items,
    autoencoder=autoencoder,
    device=device,
)

Generate/Train Cycles: 0it [00:00, ?it/s]

Generate Activations:   0%|          | 0/1000000 [00:00<?, ?it/s]

Train Autoencoder:   0%|          | 0/1000000 [00:00<?, ?it/s]

/Users/alan/Documents/Repos/sparse_autoencoder/.venv/lib/python3.11/site-packages/torch/autograd/__init__.py:251: UserWarning: The operator 'aten::sgn.out' is not currently supported on the MPS backend and will fall back to run on the CPU. This may have performance implications. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/mps/MPSFallback.mm:13.)
  Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


Generate Activations:   0%|          | 0/1000000 [00:00<?, ?it/s]

Train Autoencoder:   0%|          | 0/1000000 [00:00<?, ?it/s]